## Artificial Neural Networks and Deep Learning  
##Assignment 3.3 - Self-attention and Transformers

Prof. Dr. Ir. Johan A. K. Suykens     

In this file, we first understand the self-attention mechanism by implementing it both with ``NumPy`` and ``PyTorch``.
Then, we implement a 6-layer Vision Transformer (ViT) and train it on the MNIST dataset.

All training will be conducted on a single T4 GPU.


In [ ]:
# Please first load your google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Please go to Edit > Notebook settings > Hardware accelerator > choose "T4 GPU"
# Now check if you have loaded the GPU successfully
!nvidia-smi

# Self-attention Mechanism
Self-attention is the core mechanism in Transformer.

## Self-attention with NumPy
To have a better understanding of it, we first manually implement self-attention mechanism with ``numpy``. You can check the dimension of each variable during the matrix computation.

Feel free to change the dimensions of each variable and see how the output dimension will change accordingly.

In [ ]:
import math
import numpy as np
from numpy.random import randn

# I. Define the input data X
# X consists out of 32 samples, each sample has dimensionality 256
n = 32
d = 256
X = randn(n, d) # (32, 256)

# II. Generate the projection weights
Wq = randn(d, d) #(256, 256)
Wk = randn(d, d)
Wv = randn(d, d)

# III. Project X to find its query, keys and values vectors
Q = np.dot(X, Wq) # (32, 256)
K = np.dot(X, Wk)
V = np.dot(X, Wv)

# IV. Compute the self-attention score, denoted by A
# A = softmax(QK^T / \sqrt{d})
# Define the softmax function
def softmax(z):
    z = np.clip(z, 100, -100) # clip in case softmax explodes
    tmp = np.exp(z)
    res = np.exp(z) / np.sum(tmp, axis=1)
    return res

A = softmax(np.dot(Q, K.transpose())/math.sqrt(d)) #(32, 32)

# V. Compute the self-attention output
# outputs = A * V
outputs = np.dot(A, V) #(32, 256)

print("The attention outputs are\n {}".format(outputs))

The attention outputs are
 [[ 0.5220374   1.86564986 -0.80890358 ...  1.10937697 -3.07645862
  -3.57747067]
 [ 0.5220374   1.86564986 -0.80890358 ...  1.10937697 -3.07645862
  -3.57747067]
 [ 0.5220374   1.86564986 -0.80890358 ...  1.10937697 -3.07645862
  -3.57747067]
 ...
 [ 0.5220374   1.86564986 -0.80890358 ...  1.10937697 -3.07645862
  -3.57747067]
 [ 0.5220374   1.86564986 -0.80890358 ...  1.10937697 -3.07645862
  -3.57747067]
 [ 0.5220374   1.86564986 -0.80890358 ...  1.10937697 -3.07645862
  -3.57747067]]


## Self-attention with PyTorch
Now, we implement self-attention with ``PyTorch``, which is commonly used when building Transformers.

Feel free to change the dimensions of each variable and see how the output dimension will change accordingly.

In [ ]:
import math
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, dim_input, dim_q, dim_v):
        '''
        dim_input: the dimension of each sample
        dim_q: dimension of Q matrix, should be equal to dim_k
        dim_v: dimension of V matrix, also the  dimension of the attention output
        '''
        super(SelfAttention, self).__init__()

        self.dim_input = dim_input
        self.dim_q = dim_q
        self.dim_k = dim_q
        self.dim_v = dim_v

        # Define the linear projection
        self.linear_q = nn.Linear(self.dim_input, self.dim_q, bias=False)
        self.linear_k = nn.Linear(self.dim_input, self.dim_k, bias=False)
        self.linear_v = nn.Linear(self.dim_input, self.dim_v, bias=False)
        self._norm_fact = 1 / math.sqrt(self.dim_k)

    def forward(self, x):
        batch, n, dim_q = x.shape

        q = self.linear_q(x) # (batchsize, seq_len, dim_q)
        k = self.linear_k(x) # (batchsize, seq_len, dim_k)
        v = self.linear_v(x) # (batchsize, seq_len, dim_v)
        print(f'x.shape:{x.shape} \n Q.shape:{q.shape} \n K.shape:{k.shape} \n V.shape:{v.shape}')

        dist = torch.bmm(q, k.transpose(1,2)) * self._norm_fact
        dist = torch.softmax(dist, dim=-1)
        print('attention matrix: ', dist.shape)

        outputs = torch.bmm(dist, v)
        print('attention outputs: ', outputs.shape)

        return outputs


batch_size = 32 # number of samples in a batch
dim_input = 128 # dimension of each item in the sample sequence
seq_len = 20 # sequence length for each sample
x = torch.randn(batch_size, seq_len, dim_input)
self_attention = SelfAttention(dim_input, dim_q = 64, dim_v = 32)

attention = self_attention(x)

print(attention)

x.shape:torch.Size([32, 20, 128]) 
 Q.shape:torch.Size([32, 20, 64]) 
 K.shape:torch.Size([32, 20, 64]) 
 V.shape:torch.Size([32, 20, 32])
attention matrix:  torch.Size([32, 20, 20])
attention outputs:  torch.Size([32, 20, 32])
tensor([[[ 1.2748e-01,  3.2062e-01, -9.0824e-02,  ...,  4.8949e-02,
          -8.9586e-02,  5.2573e-02],
         [ 6.9496e-02,  2.3644e-01, -1.2103e-01,  ..., -1.9161e-02,
          -1.0948e-01,  9.4250e-02],
         [ 1.3117e-01,  1.9833e-01, -1.0276e-01,  ...,  8.9701e-04,
          -4.4238e-02,  7.4584e-02],
         ...,
         [ 1.0973e-01,  2.5184e-01, -8.4412e-02,  ...,  7.0247e-03,
           6.8269e-03,  5.5296e-02],
         [ 5.0689e-02,  2.2151e-01, -8.5633e-02,  ..., -5.7543e-03,
          -4.0312e-02,  9.8725e-02],
         [ 1.8154e-01,  1.9808e-01, -4.8706e-02,  ..., -4.9543e-02,
          -9.0759e-03,  7.9488e-02]],

        [[-2.6528e-02, -1.7695e-01,  1.5042e-01,  ...,  5.2018e-02,
           5.7282e-02, -1.5907e-02],
         [-7.0393e-02

# Transformers
In this section, we implement a 6-layer Vision Transformer (ViT) and trained it on the MNIST dataset.
We consider the classification tasks.
First, we load the MNIST dataset as follows:

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, utils
from torchvision.datasets import MNIST

def get_mnist_loader(batch_size=100, shuffle=True):
    """

    :return: train_loader, test_loader
    """
    train_dataset = MNIST(root='../data',
                          train=True,
                          transform=torchvision.transforms.ToTensor(),
                          download=True)
    test_dataset = MNIST(root='../data',
                         train=False,
                         transform=torchvision.transforms.ToTensor(),
                         download=True)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=shuffle)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size,
                                              shuffle=False)
    return train_loader, test_loader

In [ ]:
# This package is needed to build the transformer
!pip install einops

## Build ViT from scratch
Recall that each Transformer block include 2 modules: the self-attention module, the feedforward module.

In [ ]:
from einops import rearrange

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) + x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads=8):
        super().__init__()
        self.heads = heads
        self.scale = dim ** -0.5

        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)
        self.to_out = nn.Linear(dim, dim)

    def forward(self, x, mask = None):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x)
        q, k, v = rearrange(qkv, 'b n (qkv h d) -> qkv b h n d', qkv=3, h=h)

        dots = torch.einsum('bhid,bhjd->bhij', q, k) * self.scale

        if mask is not None:
            mask = F.pad(mask.flatten(1), (1, 0), value = True)
            assert mask.shape[-1] == dots.shape[-1], 'mask has incorrect dimensions'
            mask = mask[:, None, :] * mask[:, :, None]
            dots.masked_fill_(~mask, float('-inf'))
            del mask

        attn = dots.softmax(dim=-1)

        out = torch.einsum('bhij,bhjd->bhid', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out =  self.to_out(out)
        return out

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Residual(PreNorm(dim, Attention(dim, heads = heads))),
                Residual(PreNorm(dim, FeedForward(dim, mlp_dim)))
            ]))

    def forward(self, x, mask=None):
        for attn, ff in self.layers:
            x = attn(x, mask=mask)
            x = ff(x)
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels=3):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2

        self.patch_size = patch_size

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.transformer = Transformer(dim, depth, heads, mlp_dim)

        self.to_cls_token = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(), # Gaussian Error Linear Units is another type of activation function
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, img, mask=None):
        p = self.patch_size

        x = rearrange(img, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=p, p2=p)
        x = self.patch_to_embedding(x)

        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        x = self.transformer(x, mask)

        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)

## Training and test function


In [ ]:
import torch.nn.functional as F

def train_epoch(model, optimizer, data_loader, loss_history):
    total_samples = len(data_loader.dataset)
    model.train()

    for i, (data, target) in enumerate(data_loader):
        data = data.cuda()
        target = target.cuda()
        optimizer.zero_grad()
        output = F.log_softmax(model(data), dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_history.append(loss.item())

In [ ]:
def evaluate(model, data_loader, loss_history):
    model.eval()

    total_samples = len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0

    # We do not need to remember the gradients when testing
    # This will help reduce memory
    with torch.no_grad():
        for data, target in data_loader:
            data = data.cuda()
            target = target.cuda()
            output = F.log_softmax(model(data), dim=1)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)

            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / total_samples
    loss_history.append(avg_loss)
    print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')

## Let's start training!
Here, you can change the ViT structure by changing the hyper-parametrs inside ``ViT`` function.
The default settings are with 6 layers, 8 heads for the multi-head attention mechanism and embedding dimension of 64.
You can also increase the number of epochs to obtain better results.

In [ ]:
import time

# You can change the architecture here
model = ViT(image_size=28, patch_size=7, num_classes=10, channels=1,
            dim=64, depth=6, heads=8, mlp_dim=128)
model = model.cuda()
# We also print the network architecture
model

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_loss_history, test_loss_history = [], []

In [ ]:
N_EPOCHS = 20

train_loader, test_loader = get_mnist_loader(batch_size=128, shuffle=True)

# Gradually reduce the learning rate while training
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

start_time = time.time()
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch,'LR:', scheduler.get_last_lr())
    train_epoch(model, optimizer, train_loader, train_loss_history)
    evaluate(model, test_loader, test_loss_history)
    scheduler.step()

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 515kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.61MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.4MB/s]


Epoch: 1 LR: [0.001]
[    0/60000 (  0%)]  Loss: 2.4613
[12800/60000 ( 21%)]  Loss: 1.0506
[25600/60000 ( 43%)]  Loss: 0.4662
[38400/60000 ( 64%)]  Loss: 0.3025
[51200/60000 ( 85%)]  Loss: 0.1466

Average test loss: 0.2259  Accuracy: 9281/10000 (92.81%)

Epoch: 2 LR: [0.00095]
[    0/60000 (  0%)]  Loss: 0.1706
[12800/60000 ( 21%)]  Loss: 0.3368
[25600/60000 ( 43%)]  Loss: 0.0974
[38400/60000 ( 64%)]  Loss: 0.2440
[51200/60000 ( 85%)]  Loss: 0.0976

Average test loss: 0.1228  Accuracy: 9602/10000 (96.02%)

Epoch: 3 LR: [0.0009025]
[    0/60000 (  0%)]  Loss: 0.1991
[12800/60000 ( 21%)]  Loss: 0.1714
[25600/60000 ( 43%)]  Loss: 0.1239
[38400/60000 ( 64%)]  Loss: 0.0754
[51200/60000 ( 85%)]  Loss: 0.0972

Average test loss: 0.0999  Accuracy: 9698/10000 (96.98%)

Epoch: 4 LR: [0.000857375]
[    0/60000 (  0%)]  Loss: 0.1090
[12800/60000 ( 21%)]  Loss: 0.0965
[25600/60000 ( 43%)]  Loss: 0.1016
[38400/60000 ( 64%)]  Loss: 0.1047
[51200/60000 ( 85%)]  Loss: 0.1229

Average test loss: 0.0875 

In [ ]:
import time
import torch
from collections import OrderedDict

# ————————————————————————————————————————————————————————————
# 1) wrap your existing loop in a function
# ————————————————————————————————————————————————————————————
def train_vit_model(params,
                    n_epochs=20,
                    batch_size=128,
                    lr=1e-3,
                    device='cuda'):
    # instantiate ViT exactly as in your baseline
    model = ViT(
        image_size=28, patch_size=7, num_classes=10, channels=1,
        dim=params['dim'], depth=params['depth'],
        heads=params['heads'], mlp_dim=params['mlp_dim']
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

    train_loss_history, test_loss_history = [], []
    train_loader, test_loader = get_mnist_loader(batch_size=batch_size, shuffle=True)

    start_time = time.time()
    for epoch in range(1, N_EPOCHS + 1):
        print('Epoch:', epoch,'LR:', scheduler.get_last_lr())
        train_epoch(model, optimizer, train_loader, train_loss_history)
        evaluate(model, test_loader, test_loss_history)
        scheduler.step()

# ————————————————————————————————————————————————————————————
# 2) define your grid of architectures (including your baseline)
# ————————————————————————————————————————————————————————————
param_grid = OrderedDict([
    ("baseline", {"dim": 64,  "depth": 6, "heads": 8,  "mlp_dim": 128}),
    ("smaller",  {"dim": 32,  "depth": 4, "heads": 4,  "mlp_dim": 64 }),
    ("wider",    {"dim": 128, "depth": 6, "heads": 8,  "mlp_dim": 256}),
    ("deeper",   {"dim": 64,  "depth": 8, "heads": 8,  "mlp_dim": 128}),
    ("combo",    {"dim": 128, "depth": 8, "heads": 16, "mlp_dim": 512}),
])

# ————————————————————————————————————————————————————————————
# 3) run all experiments and collect results
# ————————————————————————————————————————————————————————————
device = 'cuda' if torch.cuda.is_available() else 'cpu'
results = OrderedDict()

for name, params in param_grid.items():
    print(f"=== Running ViT variant: {name} ===")
    train_vit_model(params,
                          n_epochs=N_EPOCHS,
                          batch_size=128,
                          lr=0.001,
                          device=device)

=== Running ViT variant: baseline ===
Epoch: 1 LR: [0.001]
[    0/60000 (  0%)]  Loss: 2.3265
[12800/60000 ( 21%)]  Loss: 1.1083
[25600/60000 ( 43%)]  Loss: 0.3552
[38400/60000 ( 64%)]  Loss: 0.3576
[51200/60000 ( 85%)]  Loss: 0.2406

Average test loss: 0.2146  Accuracy: 9362/10000 (93.62%)

Epoch: 2 LR: [0.00095]
[    0/60000 (  0%)]  Loss: 0.1085
[12800/60000 ( 21%)]  Loss: 0.1843
[25600/60000 ( 43%)]  Loss: 0.1536
[38400/60000 ( 64%)]  Loss: 0.1311
[51200/60000 ( 85%)]  Loss: 0.1128

Average test loss: 0.1360  Accuracy: 9567/10000 (95.67%)

Epoch: 3 LR: [0.0009025]
[    0/60000 (  0%)]  Loss: 0.1703
[12800/60000 ( 21%)]  Loss: 0.1683
[25600/60000 ( 43%)]  Loss: 0.2574
[38400/60000 ( 64%)]  Loss: 0.0948
[51200/60000 ( 85%)]  Loss: 0.1614

Average test loss: 0.1169  Accuracy: 9622/10000 (96.22%)

Epoch: 4 LR: [0.000857375]
[    0/60000 (  0%)]  Loss: 0.0932
[12800/60000 ( 21%)]  Loss: 0.1381
[25600/60000 ( 43%)]  Loss: 0.0532
[38400/60000 ( 64%)]  Loss: 0.0978
[51200/60000 ( 85%)]  Lo